In [ ]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext(appName="RDD_nb")

Wczytujemy plik RDD.txt

Spark wczytuje każdą linije jako osobny wiersz jako osbny string.
```
0,1,2,3,4
5,6,7,8,9
10,11,12,13,14
15,16,17,18,19
```

In [ ]:
RDDfile = sc.textFile("RDD.txt")

In [ ]:
RDDfile.collect()

In [ ]:
RDDfile.take(1)

In [ ]:
RDDfile.take(5)

In [ ]:
RDD_flat = RDDfile.flatMap(lambda x: x.split(",")) # pokazac jak wyglada z map

In [ ]:
RDD_flat.collect()

## Transformacje

### map(fun)

Funkcja `map` w RDD umożliwia zastosowanie podanej funkcji do każdego elementu RDD, zwracając nowe RDD z przekształconymi elementami. Każdy element wejściowy jest przetwarzany niezależnie, a wynikowa liczba elementów jest taka sama jak w oryginalnym RDD.

Funkcje anonimowe lambda pozwalają na szybkie definiowanie prostych operacji bez potrzeby tworzenia osobnych funkcji. W transformacjach RDD, takich jak `map`, `flatMap` czy `filter`, umożliwiają zwięzłe przekazanie logiki przetwarzania każdego elementu RDD, co upraszcza i skraca kod.

In [ ]:
RDDmap = RDD_flat.map(lambda x: int(x))
RDDmap.collect()

In [ ]:
def fun(x):
    return int(x)

In [ ]:
RDDmap1 = RDD_flat.map(fun)
RDDmap1.collect()

In [ ]:
RDDmap.map(lambda x: x + 1).collect()

In [ ]:
# RDDmap zawiere wcześniejsze transformacje
RDDmap.collect()

In [ ]:
#Klucz wartość => dzięki posiadaniu klucza możemy wykorzystać funkcję jak groupBy join
RDDmap.map(lambda x: (x,1)).collect()

In [ ]:
RDDlist = sc.parallelize([list(range(5)),list(range(5,10)),list(range(10,15)),list(range(15,20))])
RDDlist.collect()

In [ ]:
RDDlist.map(sum).collect()

In [ ]:
RDDlist.map(len).collect()

Poniższa komenda zwiększa każdy element w każdej liście znajdującej się w RDDlist o 1, a następnie zwraca wynik jako listę list.

https://www.datacamp.com/community/tutorials/python-list-comprehension

In [ ]:
RDDlist.map(lambda x: [y + 1 for y in x]).collect()

#### ZADANIE 

>  W RDDlist wyświetl tylko parzyste liczby<br>
>  **Hint** lista składowa z warunkiem **if** (dzielenie -> %)

In [ ]:
RDDlist.map(lambda x: [y for y in x if y%2 == 0]).collect()

Wynik:
```
[[0, 2, 4], [6, 8], [10, 12, 14], [16, 18]]
```


### flatMap(func)

**flatMap()** to funkcja transformująca, która działa jak **map()**, ale "spłaszcza" wynik.

| Funkcja     | Co robi                                                      |
| ----------- | ------------------------------------------------------------ |
| `map()`     | Zwraca **jeden element** na wejście (lub listę jako element) |
| `flatMap()` | Zwraca **wiele elementów** na wejście i je **spłaszcza**   |


Funkcja `flatMap`

1) zwraca nowe RDD po zastosowaniu podanej funkcji na każdym elemencie oryginalnego RDD 
2) spłaszcza wynik |


In [ ]:
RDDlist.collect()

In [ ]:
RDDlist.flatMap(lambda x:x).collect()

>  Jaki wynik otrzymamny po wykonaniu poniższej funkcji ?

In [ ]:
RDDlist.flatMap(lambda x:x *2).collect()

In [ ]:
RDDlist.flatMap(lambda lst:[ele_lst*2 for ele_lst in lst]).collect()  # pokazac jak wyglada z map

Jednym z zastosowań funkcji **flatMapa** jest rozwijanie zagnieżdżonych struktur (np. listy w listach)

In [ ]:
RDDlist_nested = sc.parallelize([[[[0],[1]],[[2],[3]]],[[[4],[5]],[[6],[7]]]])
RDDlist_nested.collect()

In [ ]:
RDDlist_nested.flatMap(lambda x:x).collect()

In [ ]:
RDDlist_nested.flatMap(lambda x:x).flatMap(lambda x:x).collect()

In [ ]:
RDDlist_nested.flatMap(lambda x:x).flatMap(lambda x:x).flatMap(lambda x:x).collect()

#### Zadanie

>  Osiągnij taki sam wynik jak w komórce powyżej używając na RDDlist_nested: 1 x map i 2 x flatMap


**Wynik:**

```
[0, 1, 2, 3, 4, 5, 6, 7]
```

### mapValues(func)

Funkcja mapValues() działa tylko na RDD typu pary klucz-wartość (czyli (K, V)) i aplikuje funkcję tylko do wartości (V), zostawiając klucz (K) bez zmian.

**Składnia:**
```
rdd.mapValues(funkcja_na_wartości)


In [ ]:
rdd = sc.parallelize([("a", 1), ("b", 2), ("a", 3)])

In [ ]:
# Dodaj 10 do każdej wartości, ale zostaw klucze
wynik = rdd.mapValues(lambda x: x + 10)

print(wynik.collect())

#### Zadanie

>  policz średnią ocen dla poniższego rdd


In [ ]:
rdd = sc.parallelize([
    ("Ola", [("matematyka", 4), ("historia", 5), ("biologia", 3)]),
    ("Piotr", [("matematyka", 2), ("historia", 3)]),
    ("Anna", [("historia", 5), ("biologia", 4), ("chemia", 4)])
])

**Przykładowy wynik:**

```
[('Ola', 4.0), ('Piotr', 2.5), ('Anna', 4.33)]
[('Ola', {'srednia': 4.0}), ('Piotr', {'srednia': 2.5}), ('Anna', {'srednia': 4.33})]
```

### flatMapValues(func)

Funkcja flatMapValues() działa na RDD z parami (klucz, wartość) 
- działa jak mapValues()
- spłaszcza” wynik funkcji

**Składnia:**
```
rdd.flatMapValues(f)```

Działa jak:
```
rdd.map(lambda (k, v): [(k, w) for w in f(v)]).flatMap(...)




In [ ]:
rdd = sc.parallelize([
    ("Ala", ["kot", "pies"]),
    ("Ola", ["ryba"]),
    ("Ela", ["ptak", "mysz", "żaba"])
])

wynik = rdd.flatMapValues(lambda lista: lista)

print(wynik.collect())

### keys(), values()

Metody te tworzą nowe RDD odpowiednio z kluczy i wartości oryginalnego RDD (klucz, wartość)

In [ ]:
rdd.keys().collect()

In [ ]:
rdd.values().collect()

### filter(func)

Funkcja filter() służy do filtrowania danych w RDD — zachowuje tylko te elementy, dla których podana funkcja zwraca True.

**Składnia:**
```
rdd.filter(lambda x: warunek)


In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5, 6])

# Zachowaj tylko liczby większe od 2
wynik = rdd.filter(lambda x: x > 2 )

print(wynik.collect())

#### Zadanie

> Policz średnią ocen dla wszystkich uczniów oprócz Piotra.

In [ ]:
rdd = sc.parallelize([
    ("Ola", [("matematyka", 4), ("historia", 5), ("biologia", 3)]),
    ("Piotr", [("matematyka", 2), ("historia", 3)]),
    ("Anna", [("historia", 5), ("biologia", 4), ("chemia", 4)])
])

**Przykładowy wynik:**
```
[('Ola', 4.0), ('Anna', 4.33)]
```

### join(func)

> Funkcja **join()** w PySpark służy do łączenia dwóch RDD, które są zbudowane z par klucz–wartość ((key, value)), na zasadzie inner join (łączenie po wspólnych kluczach).

**Składnia:**
```
rdd1.join(rdd2)


In [ ]:
# Dane 1: uczniowie i ich ID
rdd1 = sc.parallelize([
    (1, "Anna"),
    (2, "Bartek"),
    (3, "Celina")
])

In [ ]:
# Dane 2: ID ucznia i jego ocena
rdd2 = sc.parallelize([
    (1, 5),
    (2, 3),
    (4, 4)
])

In [ ]:
# Join po ID ucznia
wynik = rdd1.join(rdd2)

print(wynik.collect())

### union(func)

> Funkcja **union()** w PySpark łączy dwa RDD tego samego typu w jeden — zwraca nowy RDD zawierający wszystkie elementy z obu RDD (jak konkatenacja).

**Składnia:**
```
rdd1.union(rdd2)


In [ ]:
rdd1 = sc.parallelize([1, 2, 3])
rdd2 = sc.parallelize([3, 4, 5])

polaczone = rdd1.union(rdd2)

print(polaczone.collect())

In [ ]:
rdd1 = sc.parallelize([("Anna", 5), ("Bartek", 4)])
rdd2 = sc.parallelize([("Anna", 5), ("Anna", 3), ("Celina", 4)])

polaczone = rdd1.union(rdd2)
print(polaczone.collect())

Jeśli chcesz usunąć duplikaty po **union()**, użyj **.distinct()**:

In [ ]:
polaczone = rdd1.union(rdd2).distinct()
print(polaczone.collect())

### distinct(func)

>Funkcja distinct() w PySparku:<br>
    - nie przyjmuje żadnych argumentów (w tym func)<br>
    - służy do **usuwania duplikatów** z RDD<br>
    - działa globalnie – na całych elementach (nie na kolumnach czy wybranych polach)

**Składnia:**
```
rdd.distinct()


In [ ]:
polaczone.distinct().collect()

### groupBy(func)

Metoda groupBy(func) w PySpark RDD grupuje elementy według klucza zwróconego przez funkcję func.

Zwraca RDD o strukturze (klucz, iterable), gdzie każdy klucz odpowiada grupie wartości, które dały ten sam wynik funkcji

```
rdd.groupBy(lambda x: <warunek grupujący>)
```

In [ ]:
RDDlist.flatMap(lambda x:x).collect()

In [ ]:
# Grupowanie liczb według parzystości; reszty z dzielenia
RDDlist.flatMap(lambda x:x).groupBy(lambda x: x % 2).collect()

In [ ]:
# pyspark.resultiterable.ResultIterable - specjalny obiekt Spark, który reprezentuje grupę wartości przypisanych do tego klucza

In [ ]:
RDDlist.flatMap(lambda x:x).groupBy(lambda x: x % 2).mapValues(list).collect()

#### Zadanie

Pogrupuj uczniów po pierwszej literze imienia.<br>
**Hint** wykorzystaj `mapValues()`

In [ ]:
rdd = sc.parallelize(["Anna", "Ala", "Bartek", "Beata", "Celina"])

**Przykładowy wynik**

```
[('B', ['Bartek', 'Beata']), ('C', ['Celina']), ('A', ['Anna', 'Ala'])]
```

### groupByKey(func)

> Metoda groupByKey() służy do grupowania wartości RDD typu (klucz, wartość) według kluczy.

```
(key, iterable(values))
```

In [ ]:
rdd = sc.parallelize([
    ("matematyka", 5),
    ("biologia", 4),
    ("matematyka", 3),
    ("historia", 4),
    ("biologia", 5)
])

In [ ]:
grupowane = rdd.groupByKey()

In [ ]:
print(grupowane.mapValues(list).collect())

| Metoda         | Kiedy używać?                                               |
| -------------- | ----------------------------------------------------------- |
| `groupBy()`    | Gdy potrzebujesz grupować po warunku / regule               |
| `groupByKey()` | Gdy RDD ma formę `(key, value)` i chcesz grupować po kluczu |


#### Zadanie

średnia ocen po przedmiotach dla `rdd`

**Przykładowy wynik:**

```
[('historia', 4.0), ('matematyka', 4.0), ('biologia', 4.5)]
```

### reduceByKey(func)

Metoda **reduceByKey(func)** agreguje wartości o tych samych kluczach za pomocą funkcji redukującej func.

**Składnia**

```
rdd.reduceByKey(lambda x, y: <operacja_na_wartościach>)


| Metoda          | Plusy                                          | Minusy                                                                       |
| --------------- | ---------------------------------------------- | ---------------------------------------------------------------------------- |
| `groupByKey()`  | Prosta do grupowania                           | Mniej wydajna – przesyła wszystkie dane                                      |
| `reduceByKey()` | Wydajniejsza – agreguje lokalnie przed shuffle | Nie nadaje się do złożonych grupowań (np. średnia – wymaga dodatkowego kodu) |


In [ ]:
rdd.collect()

In [ ]:
suma_ocen = rdd.reduceByKey(lambda x, y: x + y)

print(suma_ocen.collect())

### aggregateByKey()

Metoda aggregateByKey(zeroValue, seqFunc, combFunc) umożliwia elastyczne agregowanie danych po kluczach, z osobnymi funkcjami dla:

agregacji lokalnej (na partycji): seqFunc

łączenia wyników z różnych partycji: combFunc

**Składnia**
```
rdd.aggregateByKey(zeroValue, seqFunc, combFunc)


- zeroValue: wartość początkowa (np. (0, 0) dla sumy i licznika)

- seqFunc: funkcja lokalna, działa w ramach partycji

- combFunc: funkcja globalna, łączy wyniki z różnych partycji

In [ ]:
rdd.collect()

In [ ]:
zero_value = (0, 0)  # (suma, licznik)

# seqFunc = dodajemy ocenę lokalnie
# combFunc = sumujemy wyniki między partycjami
srednie = rdd.aggregateByKey(
    zero_value,
    lambda acc, x: (acc[0] + x, acc[1] + 1),    # lokalnie w partycji; dodaje nową ocenę x do sumy i zwiększa licznik o 1.
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])  # między partycjami; scala dwa wyniki cząstkowe (sumy i liczniki) z różnych partycji
)
print(srednie.collect())

srednie_final = srednie.mapValues(lambda x: round(x[0] / x[1], 2))

print(srednie_final.collect())

## Akcje

**_UWAGA: `collect`, `take` i ich wariacje to niebezpieczne akcje - mogą doprowadzić do zapchania drivera i przerwania działania aplikacji, zachowaj ostrożność_**

### collect()

Zwraca elementy zbioru na driver.

In [ ]:
RDDlist.collect()

### collectAsMap()

Metoda collectAsMap() zbiera zawartość RDD (typu (klucz, wartość)) i zwraca jako zwykły słownik Pythona (dict).

In [ ]:
rdd.collectAsMap()

### take(n)

Zwraca `n` pierwszych elementów zbioru na driver.

In [ ]:
RDDlist.take(2)

### takeOrdered(n, [key])

Zwraca `n` pierwszych elementów zbioru stosując naturalny porządek lub inny wskazany.

In [ ]:
rdd.collect()

In [ ]:
# zwraca n najmniejszych elementów z RDD; malejąco (negujemy -> 5 -> -5)
rdd.takeOrdered(3, (lambda x: -x[1]))

### first()

Zwraca pierwszy element zbioru. Podobne do `take(1)`.

In [ ]:
rdd.first()

### count()

Zwraca liczbę elementów w zbiorze.

In [ ]:
rdd.count()

### sum()

Zwraca sumę elementów w RDD

In [ ]:
RDDmap.collect()

In [ ]:
RDDmap.sum()

### countByKey()

Metoda countByKey() liczy ile razy każdy klucz występuje w RDD typu (klucz, wartość).

**Składnia:**
```
rdd.countByKey()
```

***Alternatywa z RDD:***
```
rdd.mapValues(lambda x: 1).reduceByKey(lambda a, b: a + b).collect()```



In [ ]:
rdd = sc.parallelize([
    ("matematyka", 5),
    ("biologia", 4),
    ("matematyka", 3),
    ("historia", 4),
    ("biologia", 5),
    ("biologia", 2)
])

wynik = rdd.countByKey()

print(wynik)

### reduce(func)

Metoda reduce(func) w PySpark **łączy wszystkie elementy RDD w jedną wartość**, stosując podaną funkcję **redukującą parę elementów**.
Działa jak functools.reduce() w Pythonie, ale działa **równolegle** na klastry Spark (niezaleznie na partycjach).

**Składnia:**
```
rdd.reduce(lambda a, b: <operacja>)


In [ ]:
RDD_flat.collect()

In [ ]:
result = RDD_flat.reduce(lambda a, b: a + b)

print(result)

#### Zadanie

Zsumuj  `RDD_flat`

**Wynik**
`190`

### max(), mean(), min(), stdev(), variance(), stats()

In [ ]:
RDDlist.flatMap(lambda x:x).max()

In [ ]:
RDDlist.flatMap(lambda x:x).mean()

In [ ]:
RDDlist.flatMap(lambda x:x).min()

In [ ]:
RDDlist.flatMap(lambda x:x).stdev()

In [ ]:
RDDlist.flatMap(lambda x:x).variance()

In [ ]:
RDDlist.flatMap(lambda x:x).stats()